In [2]:
import pandas as pd
import pyodbc

In [3]:
# Connect to DataBank
connection_string = (
    "DSN=DataBank;"
    "Trusted_Connection=yes;"
)

con = pyodbc.connect(connection_string)

In [4]:
spend = pd.read_sql("SELECT * FROM ukri_ssd_snapshot.2025_01_09_ssd_spend;", con)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_30460\684459653.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  spend = pd.read_sql("SELECT * FROM ukri_ssd_snapshot.2025_01_09_ssd_spend;", con)


In [6]:
dim_opportunity = pd.read_sql("SELECT * FROM databank.dim_opportunity;", con)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_30460\1554799385.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_opportunity = pd.read_sql("SELECT * FROM databank.dim_opportunity;", con)


In [7]:
dim_opportunity.drop_duplicates(subset=['OpportunityName', 'OpportunityNumber', 'OpportunityOriginSource'], inplace=True)

In [9]:
df_application = pd.read_sql("SELECT * FROM ukri_ssd_snapshot.2025_01_09_ssd_application;", con)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_30460\108949258.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_application = pd.read_sql("SELECT * FROM ukri_ssd_snapshot.2025_01_09_ssd_application;", con)


In [11]:
data_merge = pd.merge(df_application, dim_opportunity, left_on=['opportunity_name', 'opportunity_id', 'application_source'],
                      right_on=['OpportunityName', 'OpportunityNumber', 'OpportunityOriginSource'], how='left', indicator=True)

In [12]:
data_merge.drop(columns=['_merge', 'application_status', 'derived_status', 'decision_date', 'lead_organisation_department_id', 'rejection_reason', 
                        'date_last_received', 'OpportunityStartDate', 'OpportunityEndDate', 'OpportunityCloseDate', 'international_programme', 'OpportunitySKey',
                        'OpportunityName', 'parent_award', 'parent_award_reference', 'redact_abstract_on_gtr', 'application_stage', 'application_stage_status',
                        'joint_proposal', 'OpportunityNumber', 'OpportunityOriginSource', 'publish_to_gtr'], inplace=True)

In [14]:
df_application[['application_reference', 'application_source']].duplicated().sum()

19

In [15]:
data_w_opp = pd.merge(spend, data_merge.drop_duplicates(subset=['application_reference', 'application_source'], keep=False), 
                      on=['application_reference', 'application_source'])

In [17]:
data_app = data_w_opp[data_w_opp['ApplicantLedOrTargeted'] == 'Applicant-Led']


In [22]:
data_app[data_app['application_category'] == 'Research Grant'].groupby(['paymentdate_financial_year', 'application_category']).agg({'amount':'sum'})

,,amount
paymentdate_financial_year,application_category,
1994/95,Research Grant,1.849680e+05
1995/96,Research Grant,3.289460e+06
1996/97,Research Grant,1.527537e+07
1997/98,Research Grant,2.809381e+07
1998/99,Research Grant,4.398211e+07
1999/00,Research Grant,5.965369e+07
2000/01,Research Grant,6.820408e+07
2001/02,Research Grant,7.996408e+07
2002/03,Research Grant,9.278506e+07
